# CancerNet Model

## Importing packages 

In [1]:
import os
from imutils import paths
import random
import shutil

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import classification_report
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from imutils import paths
import numpy as np
import argparse
import easydict

os.environ["CUDA_VISIBLE_DEVICES"]="0";

## Set the path  

In [2]:
ORIG_INPUT_DATASET = "/home/haghob/Documents/ydays_project/breast_cancer_MRI/IDC_regular_ps50_idx5"
BASE_PATH = "/home/haghob/Documents/ydays_project/breast_cancer_MRI/idc"

#Derive the training, validation, and testing directories
TRAIN_PATH = os.path.sep.join([BASE_PATH, "training"])
VAL_PATH = os.path.sep.join([BASE_PATH, "validation"])
TEST_PATH = os.path.sep.join([BASE_PATH, "testing"])

#Define the amount of data that will be used training
TRAIN_SPLIT = 0.8

#The amount of validation data will be a percentage of the *training* data
VAL_SPLIT = 0.1

## Splitting 

In [3]:
#Grab the paths to all input images in the original input directory and shuffle them

imagePaths = list(paths.list_images(ORIG_INPUT_DATASET))
random.seed(42)
random.shuffle(imagePaths)

In [4]:
#Compute the training and testing split

i = int(len(imagePaths) * TRAIN_SPLIT)
trainPaths = imagePaths[:i]
testPaths = imagePaths[i:]

In [5]:
#We'll be using part of the training data for validation

i = int(len(trainPaths) * VAL_SPLIT)
valPaths = trainPaths[:i]
trainPaths = trainPaths[i:]

In [6]:
#Define the datasets that we'll be building

datasets = [("training", trainPaths, TRAIN_PATH),("validation", valPaths, VAL_PATH),
            ("testing", testPaths, TEST_PATH)] 

In [7]:
#Loop over the datasets

for (dType, imagePaths, baseOutput) in datasets:
    #Show which data split we are creating
    print("[INFO] building '{}' split".format(dType))
    
    #If the output base output directory does not exist, create it
    if not os.path.exists(baseOutput):
        print("[INFO] 'creating {}' directory".format(baseOutput))
        os.makedirs(baseOutput)
    
    #Loop over the input image paths
    for inputPath in imagePaths:
        
        #Extract the filename of the input image and extract the class label 0 for "negative" and 1 for "positive"
        filename = inputPath.split(os.path.sep)[-1]
        label = filename[-5:-4]
        
        #Build the path to the label directory
        labelPath = os.path.sep.join([baseOutput, label])
        
        # If the label output directory does not exist, create it
        if not os.path.exists(labelPath):
            print("[INFO] 'creating {}' directory".format(labelPath))
            os.makedirs(labelPath)
        
        #Construct the path to the destination image and then copy the image itself
        p = os.path.sep.join([labelPath, filename])
        shutil.copy2(inputPath, p)

[INFO] building 'training' split
[INFO] building 'validation' split
[INFO] building 'testing' split


## Model creation

In [8]:
class CancerNet:
    @staticmethod
    def build(width, height, depth, classes):
        
        #Initialize the model along with the input shape to be"channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        
        #If we are using "channels first", update the input shape and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
            
        # CONV => RELU => POOL
        model.add(SeparableConv2D(32, (3, 3), padding="same",input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        #(CONV => RELU => POOL) * 2
        model.add(SeparableConv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(SeparableConv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        # (CONV => RELU => POOL) * 3
        model.add(SeparableConv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(SeparableConv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(SeparableConv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        #First set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(256))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        #Another FC, Softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model

## Training preparation 

In [9]:
#Construct the argument parser and parse the arguments 
args = easydict.EasyDict({"p": "plot.png"}) 

#Initialize our number of epochs, initial learning rate, and batch size 
NUM_EPOCHS = 1
INIT_LR = 1e-3
BS = 32

#Determine the total number of image paths in training, validation, and testing directories
trainPaths = list(paths.list_images(TRAIN_PATH))
totalTrain = len(trainPaths)
totalVal = len(list(paths.list_images(VAL_PATH)))
totalTest = len(list(paths.list_images(TEST_PATH)))

#Calculate the total number of training images in each class and initialize a dictionary to store the class weights
trainLabels = [int(p.split(os.path.sep)[-2]) for p in trainPaths]
trainLabels = to_categorical(trainLabels)
classTotals = trainLabels.sum(axis=0)
classWeight = dict()

#Loop over all classes and calculate the class weight
for i in range(0, len(classTotals)):
    classWeight[i] = classTotals.max() / classTotals[i]
#Initialize the training data augmentation object
trainAug = ImageDataGenerator(rescale=1 / 255.0, rotation_range=20, zoom_range=0.05,
    width_shift_range=0.1, height_shift_range=0.1, shear_range=0.05, horizontal_flip=True,
    vertical_flip=True, fill_mode="nearest")

# Initialize the validation (and testing) data augmentation object
valAug = ImageDataGenerator(rescale=1 / 255.0)

#Initialize the training generator
trainGen = trainAug.flow_from_directory(TRAIN_PATH, class_mode="categorical", target_size=(48, 48),
    color_mode="rgb", shuffle=True, batch_size=BS)

#Initialize the validation generator
valGen = valAug.flow_from_directory(VAL_PATH, class_mode="categorical", target_size=(48, 48),
    color_mode="rgb",shuffle=False, batch_size=BS)

#Initialize the testing generator
testGen = valAug.flow_from_directory(TEST_PATH, class_mode="categorical", target_size=(48, 48),
    color_mode="rgb", shuffle=False, batch_size=BS)

Found 199818 images belonging to 2 classes.
Found 22201 images belonging to 2 classes.
Found 55505 images belonging to 2 classes.


## Training

In [10]:
model = CancerNet.build(width=48, height=48, depth=3,classes=2)

#Here we are testing Adagrad as optimizer
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / NUM_EPOCHS)

model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
#Fit the model

H = model.fit(x=trainGen, steps_per_epoch=totalTrain // BS,
    validation_data=valGen,validation_steps=totalVal // BS,
    class_weight=classWeight,epochs=NUM_EPOCHS)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
6244/6244 [==============================] - 815s 131ms/step - loss: 0.5915 - acc: 0.8221 - val_loss: 0.4286 - val_acc: 0.8378


In [11]:
model.save('/home/haghob/Documents/ydays_project/CancerNet.h5') 
np.save('CancerNet.npy',H.history) 

In [12]:
model = keras.models.load_model('/home/haghob/Documents/ydays_project/CancerNet.h5')
history=np.load('CancerNet.npy',allow_pickle='TRUE').item()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## Visualization 

In [13]:
# Reset the testing generator and then use our trained model to make predictions on the data
print("[INFO] evaluating network...")

testGen.reset()
predIdxs = model.predict(x=testGen, steps=(totalTest // BS) + 1)

#For each image in the testing set we need to find the index of the label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

#Show a nicely formatted classification report
print(classification_report(testGen.classes, predIdxs,target_names=testGen.class_indices.keys()))

[INFO] evaluating network...
              precision    recall  f1-score   support

           0       0.92      0.84      0.88     39624
           1       0.68      0.82      0.74     15881

    accuracy                           0.84     55505
   macro avg       0.80      0.83      0.81     55505
weighted avg       0.85      0.84      0.84     55505



In [14]:
#Compute the confusion matrix and and use it to derive the raw accuracy, sensitivity, and specificity
cm = confusion_matrix(testGen.classes, predIdxs)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])

#Show the confusion matrix, accuracy, sensitivity, and specificity
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))

[[33352  6272]
 [ 2843 13038]]
acc: 0.8358
sensitivity: 0.8417
specificity: 0.8210


In [15]:
#Plot the training loss and accuracy

N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history["acc"], label="train_acc")
plt.plot(np.arange(0, N), history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["p"])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d (SeparableC (None, 48, 48, 32)        155       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 32)        0         
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 24, 24, 64)        2400      
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 64)        0